# MLflow
## Назначение
* Отслеживание экспериментов
* Упаковка кода в воспроизводимый формат
* Хранение, управление и разворачивание моделей

# MLflow Tracking
**MLflow Tracking** -  это набор из API и UI для логирования параметров, версий кода, метрик и выходных файлов для ваших моделей машинного обучения, а также для визуализации результатов

In [2]:
import mlflow

In [ ]:
mlflow?

### Давайте что-нибудь залогируем!

In [ ]:
mlflow.log_param?

In [ ]:
mlflow.log_metric?

In [ ]:
mlflow.log_artifact?

In [7]:
mlflow.start_run(run_name='aaa hello')

<ActiveRun: >

In [24]:
mlflow.log_param("param1", "This is a param")
mlflow.log_metric("ROC AUC", 0.75)
mlflow.log_metric("ROC AUC", 0.8)
mlflow.log_metric("ROC AUC", 0.88)
with open("artifact.txt", mode="w") as f:
    f.write("This is an artifact file")
mlflow.log_artifact("artifact.txt")

### MLflow UI запускается командой `mlflow ui`

**MLflow Tracking** крутится вокруг концепции *runs*, единиц исполнения элементов работы дата саентиста. Каждый run состот из:
* Code Version
* Start & End Time
* Source
* Parameters
* Metrics
* Artifacts

### *Run* запускается автоматические, как только мы что-то начали логировать

In [9]:
mlflow.log_param("param2", "This is in the same run as param1")

'This is in the same run as param1'

### Требутся явно завершить исполнение run для запуска следующего

In [23]:
mlflow.active_run().info.run_id

'fa8f79a78c6145518bf88f2e5a39a7ec'

In [ ]:
mlflow.end_run()

In [ ]:
mlflow.active_run().info.run_id

In [ ]:
with mlflow.start_run():
    mlflow.log_metrics({"ROC AUC": 0.7})

### Несколько экземпляторов run могу быть собраны в объект *experiment*

In [11]:
experiment_id = mlflow.create_experiment("My first experiment")

In [14]:
mlflow.start_run(experiment_id=experiment_id, run_name="aaa hello")

<ActiveRun: >

In [ ]:
with mlflow.start_run(experiment_id=experiment_id):
    mlflow.log_param("param", "param-pam-pam")

### Если явно не задать нахзвание эксперимента, все упадет в "Default"

In [ ]:
with mlflow.start_run():
    mlflow.log_metric("PR AUC", 1)

### Run тоже можно поименовать

In [ ]:
with mlflow.start_run(experiment_id=experiment_id, run_name="Run with default hyperparameters"):
    mlflow.log_param("alpha", 0.01)
    mlflow.log_metric("PR AUC", 1)

### С сервером MLflow можно взаимодействовать через объект `MlflowClient`

In [ ]:
client = mlflow.tracking.MlflowClient()

In [ ]:
client

In [ ]:
experiment = client.get_experiment_by_name("My first experiment")

In [ ]:
experiment

In [ ]:
client.search_runs(experiment_ids=experiment.experiment_id, filter_string="metrics.`PR AUC` > 0.9")

### [Больше про поиск](https://www.mlflow.org/docs/latest/search-runs.html)

### MLflow tracking server состоит из двух компонентов:
* backend store
* artifact store

### Компонент backend store это хранилище экспериментов и метаданных для запусков вместе с тегами, параметрами и метриками. Данные хранятся в файловом видел или в совместимом с SQLAlchemy хранилище. По умолчанию хранилище файловое

In [7]:
EXPERIMENT_ID = "0"

In [8]:
!ls mlruns/$EXPERIMENT_ID

5f4a995b9dbe4af583e6761df862b892  65925df04f744cbc93d6b03cdcd6c605  meta.yaml


In [9]:
RUN_ID = client.search_runs(experiment_ids=EXPERIMENT_ID)[-1].info.run_id

NameError: name 'client' is not defined

In [ ]:
!ls mlruns/$EXPERIMENT_ID/$RUN_ID/metrics/

### Компоненет artifact store обычно требует подходящего для хранения больших бинарных файлов хранилища (файловая система, hdfs, s3), куда клиент сложит свои артефакты.

In [ ]:
!ls mlruns/$EXPERIMENT_ID/$RUN_ID/artifacts

In [ ]:
!cat mlruns/$EXPERIMENT_ID/$RUN_ID/meta.yaml

### artifact store полезен для передачи модели вместе с датасетом, чтобы не приходилось все собирать с нуля

# MLflow models
### MLflow Model использует стандарный формат для упаковки моделей, чтобы его можно было просто встроить в любой программный продукт - от апишки до Spark.

In [15]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [16]:
X, y = make_classification()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(*make_classification())

In [18]:
estimator = RandomForestClassifier()
estimator.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
estimator.score(X_test, y_test)

0.68

In [30]:
pip freeze

adal==1.2.7
aiofiles==23.2.1
aiohttp==3.9.3
aiokafka==0.10.0
aiosignal==1.3.1
alembic==1.13.1
anyio==4.3.0
argcomplete==3.2.2
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
async-timeout==4.0.3
attrs==23.2.0
azure-common==1.1.28
azure-core==1.30.0
azure-graphrbac==0.61.1
azure-mgmt-authorization==4.0.0
azure-mgmt-containerregistry==10.3.0
azure-mgmt-core==1.4.0
azure-mgmt-keyvault==10.3.0
azure-mgmt-network==25.2.0
azure-mgmt-resource==23.0.1
azure-mgmt-storage==21.1.0
azureml-core==1.55.0
backports.tempfile==1.0
backports.weakref==1.0.post1
bcrypt==4.1.2
blinker==1.7.0
boto3==1.34.44
botocore==1.34.44
Bottleneck @ file:///croot/bottleneck_1707864210935/work
Brotli @ file:///tmp/abs_ecyw11_7ze/croots/recipe/brotli-split_1659616059936/work
cachetools==5.3.2
certifi @ file:///croot/certifi_1707229174982/work/certifi
cffi==1.16.0
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
click @ file:///croot/click_1698129812380/work
cloudp

In [32]:
import pandas as pd

pd.__version__

'2.1.4'

In [20]:
import mlflow.sklearn

In [ ]:
mlflow.

In [ ]:
with mlflow.start_run():
    mlflow.sklearn.log_model(estimator, artifact_path="models")

In [21]:
mlflow.sklearn.log_model(estimator, artifact_path="models")

/opt/conda/envs/dsenv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
mlflow.end_run()

### `log_model` сохраняет модель, но не отслеживает гиперпараметры. Как быть?

In [ ]:
mlflow.end_run()

In [29]:
accuracy = estimator.score(X_test, y_test)
mlflow.log_metric("Accuracy", accuracy)

In [ ]:
with mlflow.start_run():
    estimator = RandomForestClassifier()
    mlflow.log_params(estimator.get_params())
    estimator.fit(X_train, y_train)
    accuracy = estimator.score(X_test, y_test)
    mlflow.log_metric("Accuracy", accuracy)
    mlflow.sklearn.log_model(estimator, artifact_path="models")

### Некоторые имплементации моделей поддерживают [automatic logging](https://www.mlflow.org/docs/latest/tracking/tracking-api.html#id1)

In [ ]:
import xgboost
import mlflow.xgboost

In [ ]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
num_round = 3

In [ ]:
mlflow.xgboost.autolog()

In [ ]:
dtrain = xgboost.DMatrix(data=X_train, label=y_train)

In [ ]:
dtest = xgboost.DMatrix(data=X_test, label=y_test)

In [ ]:
with mlflow.start_run():
    bst = xgboost.train(param, dtrain, num_round)

In [ ]:
estimator = RandomForestClassifier()

In [ ]:
mlflow.sklearn.autolog()

In [ ]:
with mlflow.start_run():
    estimator.fit(X_train, y_train)

### Модель можно даже скормить Spark UDF.

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

In [ ]:
data = load_iris(as_frame=True)

In [ ]:
pdf = data["frame"]
target = pdf.pop("target")

In [ ]:
pdf.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression())
])

In [ ]:
pdf_train, pdf_test, target_train, target_test = train_test_split(pdf, target)

In [ ]:
experiment_id = mlflow.set_experiment("Iris with sklearn")

In [ ]:
with mlflow.start_run(run_name="The run I need"):
    run_id = mlflow.active_run().info.run_id
    print(run_id)
    pdf_train.to_pickle("dataset_train.pickle")
    mlflow.log_artifact("dataset_train.pickle")
    pipeline.fit(pdf_train, target_train)
    mlflow.sklearn.log_model(pipeline, "model")

In [ ]:
logged_model = f'runs:/{run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(pdf_test)

### А теперь Spark UDF

In [ ]:
import os
import sys

SPARK_HOME = "/usr/lib/spark3"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.9.5-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.driver.memory", "4g")
conf.set("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")

spark = SparkSession.builder.config(conf=conf).appName("MLflow model inference with Spark").getOrCreate()

In [ ]:
spark

Небольшой хак из-за конфликта версий

In [ ]:
pdf_test.iteritems = pdf_test.items

In [ ]:
spark_df = spark.createDataFrame(pdf_test)

Давайте обернем нашу модель с помощью функционала mlflow.pyfunc!

Загрузите модель, которую обучили выше, сделайте из нее spark udf

In [ ]:
mlflow.pyfunc?

In [ ]:
# Ваш код здесь!
logged_model = ...
loaded_model = ...

А теперь предикты!

In [ ]:
from pyspark.sql.functions import struct

Подсказка: вам нужно передать колонки для udf как stuct-объект

In [ ]:
# Ваш код здесь!

In [ ]:
results.show(20)

In [ ]:
results.count()

А как вернуть обратно в Pandas?

In [ ]:
# Ваш код здесь!

In [ ]:
spark_df.printSchema()

In [ ]:
spark.stop()